# Python101 - part 2

### Decorators, context managers, error handling

Mentoring material for a better understanding of some python basic concepts

## Decorators

In [1]:
# just a function taking a function as argument (and usually returning a function)
def my_decorator(fn):
    def decorated_fn(*args, **kwargs):
        print('args:', args, 'kwargs:', kwargs)
        result = fn(*args, **kwargs)
        print('result:', result)
        return result
    return decorated_fn

In [2]:
# without synctactic sugar

def func1(x, y):
    print('x:', x, 'y:', y)
    return 2 * x + y

func1 = my_decorator(func1)

func1(55, y=1)

args: (55,) kwargs: {'y': 1}
x: 55 y: 1
result: 111


111

In [3]:
# with synctactic sugar (exactly the same!)

@my_decorator
def func2(x, y):
    print('x:', x, 'y:', y)
    return 2 * x + y

func2(55, y=1)

args: (55,) kwargs: {'y': 1}
x: 55 y: 1
result: 111


111

> Don't confuse the syncatic sugar and the concept

In [4]:
# but we have some issues:
print(func1.__name__)
print(func2.__name__)
print(func1)
print(func2)

decorated_fn
decorated_fn
<function my_decorator.<locals>.decorated_fn at 0x1040b6620>
<function my_decorator.<locals>.decorated_fn at 0x1040b6c80>


In [5]:
# we'll need some help!
from functools import wraps

# functools.wraps is a base util to create clean decorators!

In [6]:
# just a function taking a function as argument (and returning a function)
def my_decorator_wrapped(fn):
    @wraps(fn)
    def decorated_fn(*args, **kwargs):
        print('args:', args, 'kwargs:', kwargs)
        result = fn(*args, **kwargs)
        print('result:', result)
        return result
    return decorated_fn

@my_decorator_wrapped
def func3(x, y):
    """
    Solves all your problems.
    Doubles everything.
    """
    print('x:', x, 'y:', y)
    return 2 * x + y

# now we're fine!
print(func3.__name__)
print(func3)

func3
<function func3 at 0x1040b6840>


In [7]:
# also keeps the docstring untouched:
func3.__doc__

'\n    Solves all your problems.\n    Doubles everything.\n    '

In [8]:
# the behaviour is unchanged
func3(32, y=1)

args: (32,) kwargs: {'y': 1}
x: 32 y: 1
result: 65


65

### Decorators != Decorator factories

In [9]:
# a decorator: takes a function as parameter

@my_decorator_wrapped
def func4():
    pass

In [10]:
# a decorator factory: takes parameters and returns a decorator

def my_decorator_factory(name):
    def my_decorator(fn):
        @wraps(fn)
        def decorated_fn(*args, **kwargs):
            print('before:', name)
            result = fn(*args, **kwargs)
            print('after:', name)
            return result
        return decorated_fn
    return my_decorator

@my_decorator_factory('my-name')
def func5():
    print('fn')

func5()

before: my-name
fn
after: my-name


> `my_decorator_factory` is a decorator factory

> `my_decorator_factory('my-name')` is a decorator

> `my_decorator_factory('my-name')(some_function)` is a decorated function

### Decorators don't have to return functions!

In [11]:
# "fun" example (don't use this!):

def iife(fn):
    """
    JS-like IIFE (Immediately Invoked Function Expressions)
    """
    return fn()

@iife
def toto():
    print('preparing...')
    return 'toto'.upper()

toto  # = iife(toto) = toto()

preparing...


'TOTO'

## Context managers

In [12]:
# typical use case: manage resources (files, db connections...)
# (think "defer" in golang, "finally" in JS/python)

def crash():
    print('> before')
    
    try:
        crash_because_not_defined()
    finally:
        print('> finally')  # will be called

    print('> after')  # will not be called!

try:
    crash()
except NameError as err:
    print(err)

> before
> finally
name 'crash_because_not_defined' is not defined


In [13]:
# can be implemented with a class implementing two methods: __enter__ & __exit__

class MyContextManager:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        print(f'> enter {self.name}')
        return self.name.upper()

    def __exit__(self, *args):
        print(f'> exit {self.name}')


    
def crash():
    print('> before')
    
    with MyContextManager('the danger zone') as cm:
        print(f'> got cm: {cm}')
        crash_because_not_defined()

    print('> after')  # will not be called!

try:
    crash()
except NameError as err:
    print(err)

> before
> enter the danger zone
> got cm: THE DANGER ZONE
> exit the danger zone
name 'crash_because_not_defined' is not defined


In [14]:
# real world example: open a file

emails = ['pierre@gmail.com', 'pierre@hotmail.com', 'pierre@yopmail.com', 'paul@gmail.com']

with open('/tmp/python_rocks.txt', 'w') as my_file:
    for email in emails:
        print(email, file=my_file)


with open('/tmp/python_rocks.txt', 'r') as my_file:
    saved = my_file.read()

saved  # "saved" still exists, but "my_file" is closed

'pierre@gmail.com\npierre@hotmail.com\npierre@yopmail.com\npaul@gmail.com\n'

In [15]:
# the file is closed outside of the context manager (you shouldn't even access the variable...)
try:
    my_file.read()
except ValueError as err:
    print(err)

I/O operation on closed file.


In [16]:
# another example: welcome tempfile!
# extract from https://docs.python.org/3.6/library/tempfile.html#examples
import tempfile
import os

with tempfile.NamedTemporaryFile() as fp:
    fp.write(b'Hello world!')
    temp_path = fp.name
    print('path:', temp_path)
    
    file_exists = os.path.isfile(temp_path)
    print('file_exists:', file_exists)
    
    fp.seek(0)
    stored = fp.read()
    print('stored:', stored)

file_exists = os.path.isfile(temp_path)
print('file_exists:', file_exists)

path: /var/folders/zv/mdh8jytn257dtlj6qknszfnm0000gp/T/tmp8jbykopw
file_exists: True
stored: b'Hello world!'
file_exists: False


## Error handling

### Be specific

Always catch the error you expect, close to where you expect it

In [17]:
def get_nested_key_safe(d: dict):
    try:
        return d['nested']['key']
    except KeyError:
        return 'default_value'

[get_nested_key_safe(d) for d in [{'nested': {'key': 55}}, {'nested': {}}, {}]]

[55, 'default_value', 'default_value']

### Let it raise

Don't try/except something that is not supposed to happen, and you can't do anything intelligent about it

In [18]:
 # if this is always supposed to be present, don't try to handle the case it isn't
def get_nested_key(d: dict):
    return d['nested']['key']

try:
    get_nested_key({})  # raises a very clear KeyError('nested')
except Exception as err:
    print(repr(err))

KeyError('nested',)


In [19]:
# the same is true for dict.get, which is a hidden try/except
# it is not always relevant and often abused

def get_nested_key_bad(d: dict):
    return d.get('nested').get('key')

try:
    get_nested_key_bad({})  # raises an impossible to exploit TypeError
except Exception as err:
    print(repr(err))

AttributeError("'NoneType' object has no attribute 'get'",)


### EAFP: Easier to Ask for Forgiveness than Permission

More on this in the awesome [Python Anti-Patterns book](https://docs.quantifiedcode.com/python-anti-patterns/readability/asking_for_permission_instead_of_forgiveness_when_working_with_files.html)

In [20]:
# bad non-idiomatic implementation

def bad_load_from_file(filename):
    if os.path.isfile(filename):
        with open(filename) as f:
            return f.read()
    else:
        print('fallbacking...')
        return 'SOME_DEFAULT'

bad_load_from_file('/tmp/doesnotexist.txt')

fallbacking...


'SOME_DEFAULT'

In [21]:
# EAFP implementation

def eafp_load_from_file(filename):
    try:
        with open(filename) as f:
            return f.read()
    except OSError:
        print('fallbacking...')
        return 'SOME_DEFAULT'

eafp_load_from_file('/tmp/doesnotexist.txt')

fallbacking...


'SOME_DEFAULT'

> Of course, not only a Python thing! This philosophy prevents race conditions!

### Custom error handling (e.g. for business logic)

In [22]:
# define your own error type inheriting from Exception:
class FooError(Exception):
    pass

def foo():
    raise FooError(500)

err = None
try:
    foo()
# catch regarding the exception types you expect:
except FooError as e:
    err = e

err

__main__.FooError(500)